In [1]:
import os
import random
from tqdm import tqdm
import numpy as np
from sklearn.metrics import f1_score
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import lightgbm as lgb
 

In [2]:
train_df = 'train.csv'
train_df = pd.read_csv(train_df, skipinitialspace=True, na_values='?')
train_df

,age,workclass,final_weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [3]:
train_df.dropna(inplace=True)
train_df.head()

,age,workclass,final_weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
test_df = pd.read_csv('test.csv')
test_df.head()

,ID,age,workclass,final_weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States
1,1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States
2,2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States
3,3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States
4,4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States


In [5]:
def load_dataset(train, test):
    
    train = pd.read_csv(train, skipinitialspace=True, na_values='?')
    test = pd.read_csv(test, skipinitialspace=True, na_values='?')
    
    income_map = {'<=50K':0, '>50K':1}
    y = train['class'].map(income_map)
    
    
    train = train.astype(str).apply(LabelEncoder().fit_transform)
    test = test.astype(str).apply(LabelEncoder().fit_transform)

    X = train.drop(['class'], axis=1)    
    test = test.drop(['ID'], axis=1)
  
    return X.values, y.values, test.values

In [6]:
train = pd.read_csv('train.csv', skipinitialspace=True, na_values='?')
test = pd.read_csv('test.csv', skipinitialspace=True, na_values='?')

print(train.isna().sum().sum())
print(test.isna().sum().sum())
income_map = {'<=50K':0, '>50K':1}
y = train['class'].map(income_map)

print("!!")

train = train.astype(str).apply(LabelEncoder().fit_transform)
print("!!")

test = test.astype(str).apply(LabelEncoder().fit_transform)

X = train.drop(['class'], axis=1)    
test = test.drop(['ID'], axis=1)

4262
2203
!!
!!


In [7]:
SEED = 7

train_path = 'train.csv'
test_path = 'test.csv'

X, y, _ = load_dataset(train_path,test_path)

VAL_RATIO = 0.2
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=VAL_RATIO, random_state=SEED)

d_train = lgb.Dataset(X_train, label=y_train)
d_val = lgb.Dataset(X_val, label=y_val)

print("train data shape : ", X_train.shape)
print("validation data shape : ", X_val.shape)

train data shape :  (26048, 14)
validation data shape :  (6513, 14)


In [8]:
model = lgb.LGBMClassifier()

print("LGBMClassifier 모델에 쓰인 parameter 값들 \n", model.get_params())

model.fit(X_train, y_train)

evals = [(X_val, y_val)]

model.fit(X_train, y_train, early_stopping_rounds=10, eval_metric="logloss", eval_set=evals, verbose = False)

LGBMClassifier 모델에 쓰인 parameter 값들 
 {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.1, 'max_depth': -1, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 100, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': None, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': 'warn', 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0}


c:\users\fmson\appdata\local\programs\python\python38\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\users\fmson\appdata\local\programs\python\python38\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


LGBMClassifier()

In [9]:
_, _, test = load_dataset(train_path,test_path)

print(test.shape)

(16281, 14)


In [10]:
y_pred = model.predict(test)

        
print(y_pred)
print(y_pred.shape)
print(y_pred.sum())

[0 0 0 ... 1 1 1]
(16281,)
3131


In [11]:
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission

,ID,class,class.1,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,0,<=50K,<=50K,O,13343.0,0.819494
1,1,<=50K,<=50K,O,NaN,NaN
2,2,<=50K,<=50K,O,NaN,NaN
3,3,<=50K,>50K,X,NaN,NaN
4,4,<=50K,<=50K,O,NaN,NaN
...,...,...,...,...,...,...
16276,16276,<=50K,<=50K,O,NaN,NaN
16277,16277,<=50K,<=50K,O,NaN,NaN
16278,16278,<=50K,>50K,X,NaN,NaN
16279,16279,<=50K,>50K,X,NaN,NaN


In [12]:
income_map = {'<=50K':0, '>50K':1}

income_map_reverse = dict(map(reversed, income_map.items()))

pred = list(map(lambda x: income_map_reverse[x], y_pred))

t = pd.read_csv(test_path)
ID = t['ID'].values

submission = pd.DataFrame({'ID':ID, 'class':pred})

submission

,ID,class
0,0,<=50K
1,1,<=50K
2,2,<=50K
3,3,>50K
4,4,<=50K
...,...,...
16276,16276,<=50K
16277,16277,<=50K
16278,16278,>50K
16279,16279,>50K


In [13]:
submission.to_csv("submission.csv", index=False)